**June 4, 2025 • 3:30 PM**

In [1]:
#!/usr/bin/env python3
# Panutad Sirikul (French)
# 650510714
# user_activity
# 229496 Sec DS

## <b>2. Plot Line Chart</b> 
รายชั่วโมง ดูว่า User สองท่านนี้ ที่โหลดแอพมาแล้วแต่ไม่ใช่งาน ได้รับการกระตุ้นให้ใช้งาน (Push Notification) แล้วตอบสนองหรือไม่

`User first booking interaction after pushing message`

user_segment_id = 138 : first called 29.05.2025 at 16:03, final called 01.06.2025 at 12:00

user_segment_id = 139 : first called 29.05.2025 at 16:03, final called 01.06.2025 at 12:00


### <b>Importing Dataset</b>

In [2]:
# importing required llibraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
# ignore warning & show all columns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)

In [4]:
# Importing Dataset
df = pd.read_csv("user_segment_raw.csv")
df

,user_id,user_segment_id,user_segment
0,392422,138,APR25 Register (Insider Campaign)
1,384365,138,APR25 Register (Insider Campaign)
2,391335,138,APR25 Register (Insider Campaign)
3,386700,138,APR25 Register (Insider Campaign)
4,391233,138,APR25 Register (Insider Campaign)
...,...,...,...
12395,396616,139,MAY25 Register (Insider Campaign)
12396,394242,139,MAY25 Register (Insider Campaign)
12397,395394,139,MAY25 Register (Insider Campaign)
12398,394216,139,MAY25 Register (Insider Campaign)


### <b>Importing data — user_first_booking.csv</b>

In [5]:
data = pd.read_csv("user_first_booking_raw.csv")
data

,user_id,registed_date,first_booking_date
0,1,2018-06-15 14:41:51,2018-07-31 18:09:36
1,2,2016-03-22 23:32:57,NaN
2,3,2018-06-15 16:54:04,2016-03-24 21:14:38
3,9,2016-03-23 14:25:07,NaN
4,10,2016-03-29 14:02:51,2018-07-31 18:12:57
...,...,...,...
401704,403911,2025-06-04 08:08:07,NaN
401705,403912,2025-06-04 08:11:06,NaN
401706,403913,2025-06-04 08:15:49,NaN
401707,403914,2025-06-04 08:17:47,NaN


### <b>Data Merging</b>

In [7]:
# Match first booking data with segment data
data = data[data['user_id'].isin(df['user_id'])]

# Rename column
data = data.rename(columns={'registed_date': 'register_date'})

# Transfer register_date and first_booking_date in data to df by user_id
df = df.merge(
    data[["user_id", "register_date", "first_booking_date"]],
    on="user_id",
    how="left"
)

df.head()

,user_id,user_segment_id,user_segment,register_date,first_booking_date
0,392422,138,APR25 Register (Insider Campaign),2025-04-28 22:41:07,NaN
1,384365,138,APR25 Register (Insider Campaign),2025-04-01 17:43:09,NaN
2,391335,138,APR25 Register (Insider Campaign),2025-04-25 23:34:02,NaN
3,386700,138,APR25 Register (Insider Campaign),2025-04-09 11:45:18,NaN
4,391233,138,APR25 Register (Insider Campaign),2025-04-25 16:47:10,NaN


## <b>Preprocessing and EDA:<b>

<div class="alert alert-block alert-info">
<b>Note:</b> Merged Dataset are somewhat clean, but I still have to deal with NaN in first_booking_date</div>

<b>Exploration on user_segment_id and user_segment</b>

In [8]:
df.user_segment_id.unique()

array([138, 139])

In [9]:
df.user_segment.unique()

array(['APR25 Register (Insider Campaign)',
       'MAY25 Register (Insider Campaign)'], dtype=object)

In [14]:
# Check if each user_segment_id has a unique user_segment in df
mapping_unique = df.groupby('user_segment_id')['user_segment'].nunique()
print(mapping_unique)
print("_________________________________________________________\n")
# Display mapping between user_segment_id and user_segment
mapping = df[['user_segment_id', 'user_segment']].drop_duplicates().sort_values('user_segment_id')
print(mapping)

user_segment_id
138    1
139    1
Name: user_segment, dtype: int64
_________________________________________________________

      user_segment_id                       user_segment
0                 138  APR25 Register (Insider Campaign)
6122              139  MAY25 Register (Insider Campaign)


In [17]:
df[(df['user_segment_id'] == 138) & (df['user_segment'] == 'MAY25 Register (Insider Campaign)')]

,user_id,user_segment_id,user_segment,register_date,first_booking_date


In [19]:
df[(df['user_segment_id'] == 139) & (df['user_segment'] == 'APR25 Register (Insider Campaign)')]

,user_id,user_segment_id,user_segment,register_date,first_booking_date


<b>General Data Exploration</b>

In [20]:
df.columns

Index(['user_id', 'user_segment_id', 'user_segment', 'register_date',
       'first_booking_date'],
      dtype='object')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12400 entries, 0 to 12399
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             12400 non-null  int64 
 1   user_segment_id     12400 non-null  int64 
 2   user_segment        12400 non-null  object
 3   register_date       12400 non-null  object
 4   first_booking_date  261 non-null    object
dtypes: int64(2), object(3)
memory usage: 484.5+ KB


In [22]:
is_unique = df['user_id'].is_unique
print("Is user_id unique?:", is_unique)

Is user_id unique?: True


<b>Data Types Transformation</b>

In [23]:
# Convert register_date and first_booking_date to datetime
df['register_date'] = pd.to_datetime(df['register_date'], errors='coerce')
df['first_booking_date'] = pd.to_datetime(df['first_booking_date'], errors='coerce')

# Ensure user_id and user_segment_id are integers
df['user_id'] = df['user_id'].astype(int)
df['user_segment_id'] = df['user_segment_id'].astype(int)

# user_segment remains as object (string)
df.dtypes

user_id                        int64
user_segment_id                int64
user_segment                  object
register_date         datetime64[ns]
first_booking_date    datetime64[ns]
dtype: object

## <b>Data Filtration</b>

In [26]:
df

,user_id,user_segment_id,user_segment,register_date,first_booking_date
0,392422,138,APR25 Register (Insider Campaign),2025-04-28 22:41:07,NaT
1,384365,138,APR25 Register (Insider Campaign),2025-04-01 17:43:09,NaT
2,391335,138,APR25 Register (Insider Campaign),2025-04-25 23:34:02,NaT
3,386700,138,APR25 Register (Insider Campaign),2025-04-09 11:45:18,NaT
4,391233,138,APR25 Register (Insider Campaign),2025-04-25 16:47:10,NaT
...,...,...,...,...,...
12395,396616,139,MAY25 Register (Insider Campaign),2025-05-12 14:12:59,NaT
12396,394242,139,MAY25 Register (Insider Campaign),2025-05-04 10:55:20,NaT
12397,395394,139,MAY25 Register (Insider Campaign),2025-05-08 05:36:12,NaT
12398,394216,139,MAY25 Register (Insider Campaign),2025-05-04 09:14:48,NaT


<b>Split Date and Time from register_date and first_booking_date</b>

In [27]:
df['register_date_only'] = df['register_date'].dt.date
df['register_time_only'] = df['register_date'].dt.time

df['first_booking_date_only'] = df['first_booking_date'].dt.date
df['first_booking_time_only'] = df['first_booking_date'].dt.time

df[['register_date_only', 'register_time_only', 'first_booking_date_only', 'first_booking_time_only']].head()

,register_date_only,register_time_only,first_booking_date_only,first_booking_time_only
0,2025-04-28,22:41:07,NaT,NaT
1,2025-04-01,17:43:09,NaT,NaT
2,2025-04-25,23:34:02,NaT,NaT
3,2025-04-09,11:45:18,NaT,NaT
4,2025-04-25,16:47:10,NaT,NaT


<b>Remove row where no response from user in first_booking_date (NaT)</b>

and save the non-response users data in another DataFrame

In [30]:
# Remove rows where first_booking_date is NaT (no response)
df_response = df[df['first_booking_date'].notna()].copy()
df_no_response = df[df['first_booking_date'].isna()].copy()

# Display the shapes of the new DataFrames
print("Users with response:", f"{df_response.shape[0]} users")
print("Users with no response:", f"{df_no_response.shape[0]} users")

Users with response: 261 users
Users with no response: 12139 users


<b>Eliminating Outliers in df_response</b>

register_date_only must be less than or equal to first_booking_date_only

In [31]:
# Filter df_response to ensure register_date_only <= first_booking_date_only
df_response = df_response[
    pd.to_datetime(df_response['register_date_only']) <= pd.to_datetime(df_response['first_booking_date_only'])
].copy()

df_response.head()

,user_id,user_segment_id,user_segment,register_date,first_booking_date,register_date_only,register_time_only,first_booking_date_only,first_booking_time_only
50,390852,138,APR25 Register (Insider Campaign),2025-04-24 14:22:43,2025-05-21 20:54:02,2025-04-24,14:22:43,2025-05-21,20:54:02
480,389939,138,APR25 Register (Insider Campaign),2025-04-21 13:05:18,2025-05-31 14:52:39,2025-04-21,13:05:18,2025-05-31,14:52:39
735,384930,138,APR25 Register (Insider Campaign),2025-04-03 13:00:47,2025-05-31 08:17:55,2025-04-03,13:00:47,2025-05-31,08:17:55
752,392508,138,APR25 Register (Insider Campaign),2025-04-29 10:06:15,2025-05-29 17:34:12,2025-04-29,10:06:15,2025-05-29,17:34:12
849,390505,138,APR25 Register (Insider Campaign),2025-04-23 14:06:37,2025-06-02 11:45:45,2025-04-23,14:06:37,2025-06-02,11:45:45


In [35]:
print(f"No changes made — {df_response.shape[0]} rows") 

No changes made — 261 rows


## <b>Find Response Duration (Day/Hour) since Push Message</b>

Metrics:

`user_segment_id = 138 : first called 29.05.2025 at 16:03`

`user_segment_id = 138 : final called 01.06.2025 at 12:00`

`user_segment_id = 139 : first called 29.05.2025 at 16:03`

`user_segment_id = 139 : final called 01.06.2025 at 12:00`

In [36]:
# Split df_response into two DataFrames by user_segment_id
df_response_138 = df_response[df_response['user_segment_id'] == 138].copy()
df_response_139 = df_response[df_response['user_segment_id'] == 139].copy()

<b>Response Days and Response Hours</b>

Negative value mean that users made their first booking before receiving the push notification, so their response time is negative.

In [44]:
from datetime import datetime

# Define push notification start and end times for both segments
push_start = datetime(2025, 5, 29, 16, 3)
push_end = datetime(2025, 6, 1, 12, 0)

# For df_response_138
df_response_138['response_timedelta'] = (
    pd.to_datetime(df_response_138['first_booking_date']) - push_start
)
df_response_138['response_days'] = df_response_138['response_timedelta'].dt.days
df_response_138['response_hours'] = df_response_138['response_timedelta'].dt.total_seconds() // 3600

# For df_response_139
df_response_139['response_timedelta'] = (
    pd.to_datetime(df_response_139['first_booking_date']) - push_start
)
df_response_139['response_days'] = df_response_139['response_timedelta'].dt.days
df_response_139['response_hours'] = df_response_139['response_timedelta'].dt.total_seconds() // 3600

# Show sample
df_response_138[['user_id', 'first_booking_date', 'response_days', 'response_hours']].head()
df_response_139[['user_id', 'first_booking_date', 'response_days', 'response_hours']].head()

,user_id,first_booking_date,response_days,response_hours
6221,401561,2025-05-30 01:27:57,0,9.0
6259,400962,2025-05-26 20:20:28,-3,-68.0
6277,399257,2025-06-03 10:01:28,4,113.0
6285,400596,2025-05-25 21:45:26,-4,-91.0
6287,398062,2025-05-27 18:13:02,-2,-46.0


<b>User Response from Push Message</b>

In [45]:
# Filter out users whose first booking was before the push notification (response_days < 0)
df_response_138 = df_response_138[df_response_138['response_days'] >= 0].copy()
df_response_139 = df_response_139[df_response_139['response_days'] >= 0].copy()

# Show sample after filtering
df_response_138[['user_id', 'first_booking_date', 'response_days', 'response_hours']].head()

,user_id,first_booking_date,response_days,response_hours
480,389939,2025-05-31 14:52:39,1,46.0
735,384930,2025-05-31 08:17:55,1,40.0
752,392508,2025-05-29 17:34:12,0,1.0
849,390505,2025-06-02 11:45:45,3,91.0
1802,386522,2025-06-02 13:12:13,3,93.0


In [46]:
df_response_139[['user_id', 'first_booking_date', 'response_days', 'response_hours']].head()

,user_id,first_booking_date,response_days,response_hours
6221,401561,2025-05-30 01:27:57,0,9.0
6277,399257,2025-06-03 10:01:28,4,113.0
6329,398018,2025-05-30 00:38:38,0,8.0
6500,393353,2025-05-31 13:27:38,1,45.0
6744,401323,2025-05-30 13:58:06,0,21.0


<b>Save User with Response's DataFrame to CSV</b>

In [47]:
df_response_138.to_csv("df_response_138.csv", index=False)
df_response_139.to_csv("df_response_139.csv", index=False)

## <b>Visualization</b>

In [50]:
print(f"Total response users in 138: {df_response_138.shape[0]} rows")
print(f"Total response users in 139: {df_response_139.shape[0]} rows")

Total response users in 138: 16 rows
Total response users in 139: 79 rows


<b>Response Days in Campaign 138 and 139</b>

In [52]:
import plotly.express as px

# Visualize response_days distribution for user_segment_id 138
fig_138 = px.histogram(
    df_response_138,
    x="response_days",
    nbins=20,
    title="Distribution of Response Days (user_segment_id = 138)",
    labels={"response_days": "Days since Push Notification", "count": "Number of Users"}
)
fig_138.show()

# Visualize response_days distribution for user_segment_id 139
fig_139 = px.histogram(
    df_response_139,
    x="response_days",
    nbins=20,
    title="Distribution of Response Days (user_segment_id = 139)",
    labels={"response_days": "Days since Push Notification", "count": "Number of Users"}
)
fig_139.show()

<b>Response Hours in Campaign 138 and 139</b>

In [53]:
# Visualize response_hours distribution for user_segment_id 138
fig_138_hours = px.histogram(
    df_response_138,
    x="response_hours",
    nbins=24,
    title="Distribution of Response Hours (user_segment_id = 138)",
    labels={"response_hours": "Hours since Push Notification", "count": "Number of Users"}
)
fig_138_hours.show()

# Visualize response_hours distribution for user_segment_id 139
fig_139_hours = px.histogram(
    df_response_139,
    x="response_hours",
    nbins=24,
    title="Distribution of Response Hours (user_segment_id = 139)",
    labels={"response_hours": "Hours since Push Notification", "count": "Number of Users"}
)
fig_139_hours.show()

<b>Finished Assignment on June 5, 2025 • 10:44 AM</b>